In [ ]:
14
Найти производителей, которые выпускают более одной модели, при этом все выпускаемые производителем модели являются продуктами одного типа. Вывести: maker, type

Select maker, type
 from product
where
  maker in (
    select maker from product
      group by maker
      having count(distinct type) = 1
  )
group by maker, type
 having
count(model) > 1


17
Найдите модели ПК-блокнотов, скорость которых меньше скорости каждого из ПК.
Вывести: type, model, speed

Select DISTINCT type, product.model, speed 
  from product, laptop 
where laptop.model = product.model 
  and speed < ALL (select speed from pc)
    
    
18
Найдите производителей самых дешевых цветных принтеров. Вывести: maker, price 
    
select distinct 
product.maker, printer.price
   from product, printer
where product.model = printer.model and
      printer.price =
      (select min( price)
          from printer
          where color = 'y') and (color = 'y')

        
20
Найдите производителей, выпускающих по меньшей мере три различных модели ПК. Вывести: Maker, число моделей ПК.
    
Select maker, count(model) 
  from product 
where type = 'pc' 
  group by maker having count(model) >= 3

    
21
Найдите максимальную цену ПК, выпускаемых каждым производителем, у которого есть модели в таблице PC.
Вывести: maker, максимальная цена.

Select maker, max(price) 
  from product inner 
    join pc on product.model = pc.model
group by maker

24
Перечислите номера моделей любых типов, имеющих самую высокую цену по всей имеющейся в базе данных продукции.

with mp as (
  select model, price from pc
  union
  select model, price from printer
  union
  select model, price from laptop
)
select model from mp where price = (select max(price) from mp)



25
Найдите производителей принтеров, которые производят ПК с наименьшим объемом RAM и с самым быстрым процессором среди всех ПК, имеющих наименьший объем RAM. Вывести: Maker

with M as (
select maker from product p
where
  model in (
    select model from pc
    where
      ram=(select min(ram) from pc)
      and speed=(
        -- max speed from pc with minimum RAN
        select max(speed) from pc
        where ram=(select min(ram) from pc)
      )
  )
)
select distinct maker from product
where type='Printer' and maker in (select maker from M)



26
Найдите среднюю цену ПК и ПК-блокнотов, выпущенных производителем A (латинская буква). Вывести: одна общая средняя цена.

with M as (
  select model, price from pc
  union all
  select model, price from laptop
)
select avg(M.price) "avg price"
  from product p join M on p.model=M.model
where p.maker='A'



27
Используя таблицу Product, определить количество производителей, выпускающих по одной модели.

select count(q.cm) "count of makers" from (
  select count(model) cm from product
  group by maker
  having count(model)=1
) q

28
В предположении, что приход и расход денег на каждом пункте приема фиксируется не чаще одного раза в день [т.е. первичный ключ (пункт, дата)], написать запрос с выходными данными (пункт, дата, приход, расход). Использовать таблицы Income_o и Outcome_o.

select
  isnull(i.point, o.point) point
  , isnull(i.date, o.date) [date]
  , inc
  , out
  from income_o i full outer join outcome_o o 
    on i.date=o.date and i.point=o.point

30
В предположении, что приход и расход денег на каждом пункте приема фиксируется произвольное число раз (первичным ключом в таблицах является столбец code), требуется получить таблицу, в которой каждому пункту за каждую дату выполнения операций будет соответствовать одна строка. Вывод: point, date, суммарный расход пункта за день (out), суммарный приход пункта за день (inc). Отсутствующие значения считать неопределенными (NULL).

select
 isnull(i.point, o.point) point
  , isnull(i.date, o.date) date
  , sum(o.out) outcome
  , sum(i.inc) income
  from income i
full join outcome o
    on i.point=o.point and i.date=o.date and i.code=o.code
group by isnull(i.point, o.point), isnull(i.date, o.date)




34
По Вашингтонскому международному договору от начала 1922 г. запрещалось строить линейные корабли водоизмещением более 35 тыс.тонн. Укажите корабли, нарушившие этот договор (учитывать только корабли c известным годом спуска на воду). Вывести названия кораблей.

Select s.name from ships s
  join classes c on s.class=c.class
where
  s.launched >= 1922
and c.displacement > 35000
and type='bb'

    
35
В таблице Product найти модели, которые состоят только из цифр или только из латинских букв (A-Z, без учета регистра). Вывод: номер модели, тип модели. (Всё это было бы проще, если бы оно умело в регулярные выражения(?)).

select model, type from product
 where model not like '%[^0-9]%' or model not like '%[^a-z]%'


37
Найдите классы, в которые входит только один корабль из базы данных (учесть также корабли в Outcomes).

select q.class from (
  select class, name from ships
  union
  select c.class, o.ship from classes c
    join outcomes o on c.class=o.ship
) q
group by q.class
having count(q.class)=1



41 
Для каждого производителя, у которого присутствуют модели хотя бы в одной из таблиц PC, Laptop или Printer, определить максимальную цену на его продукцию. Вывод:

имя производителя
если среди цен на продукцию данного производителя присутствует NULL, то выводить для этого производителя NULL, иначе максимальную цену.
with m as (
  select model, price, iif(price is null, 1, 0) pn from printer
  union all
  select model, price, iif(price is null, 1, 0) pn from pc
  union all
  select model, price, iif(price is null, 1, 0) pn from laptop
)
select
  maker
  , case
      when sum(pn) > 0 then NULL
      else max(price)
end as max_price
from product p join m on p.model=m.model
group by maker


45
Найдите названия всех кораблей в базе данных, состоящие из трех и более слов (например, King George V). Считать, что слова в названиях разделяются единичными пробелами, и нет концевых пробелов.

select name from ships where name like '_% _% _%'
union
select ship from outcomes where ship like '_% _% _%'


46
Для каждого корабля, участвовавшего в сражении при Гвадалканале (Guadalcanal), вывести название, водоизмещение и число орудий.

with allships as (
  select name, class from ships
  union
  select ship as name, ship as class from outcomes
  where ship not in (select name from ships)
)
select a.name, displacement, numGuns from allships a
  left join classes c on a.class=c.class
join outcomes o on a.name=o.ship and o.battle='Guadalcanal';


48
Найдите классы кораблей, в которых хотя бы один корабль был потоплен в сражении.

select distinct c.class from outcomes o
  left join ships s on o.ship = s.name
join classes c on (o.ship=c.class or s.class=c.class)
where result = 'sunk'


49
Найдите названия кораблей с орудиями калибра 16 дюймов (учесть корабли из таблицы Outcomes).

select name from ships s join classes c on s.class=c.class
where bore=16
union
select ship from outcomes o join classes c on o.ship=c.class
where bore=16


50
Найдите сражения, в которых участвовали корабли класса Kongo из таблицы Ships.

select distinct battle from outcomes o
  join ships s on o.ship=s.name
  join classes c on c.class=s.class
where c.class='Kongo'


51
Найдите названия кораблей, имеющих наибольшее число орудий среди всех имеющихся кораблей такого же водоизмещения (учесть корабли из таблицы Outcomes).

with sh as (
  select name, class from ships
  union
  select ship, ship from outcomes
)
select
  name
  from sh join classes c on sh.class=c.class
  where numguns >= all(
    select ci.numguns from classes ci
      where ci.displacement=c.displacement
        and ci.class in (select sh.class from sh)
    )

    
52
Определить названия всех кораблей из таблицы Ships, которые могут быть линейным японским кораблем, имеющим число главных орудий не менее девяти, калибр орудий менее 19 дюймов и водоизмещение не более 65 тыс.тонн

select
  s.name
  from ships s
  join classes c on c.class=s.class
  where country='Japan'
    and type='bb' and (numguns>=9 or numguns is null)
    and (bore<19 or bore is null)
    and (displacement<=65000 or displacement is null)

    
53
Определите среднее число орудий для классов линейных кораблей. Получить результат с точностью до 2-х десятичных знаков.

select
  cast(avg(numguns*1.0) as numeric(6,2)) as "avg numguns"
  from classes
  where type='bb'

    
54
С точностью до 2-х десятичных знаков определите среднее число орудий всех линейных кораблей (учесть корабли из таблицы Outcomes).

select
  cast(avg(numguns*1.0) as numeric(6,2)) as "avg numguns"
  from (
    select name, class from ships
    union
    select ship, ship from outcomes
  ) s
  join classes c on s.class=c.class
where type='bb'

    
55
Для каждого класса определите год, когда был спущен на воду первый корабль этого класса. Если год спуска на воду головного корабля неизвестен, определите минимальный год спуска на воду кораблей этого класса. Вывести: класс, год.

select
  c.class
  , min(launched) "launch year"
  from classes c
  full join ships s on c.class=s.class
  group by c.class

    
56
Для каждого класса определите число кораблей этого класса, потопленных в сражениях. Вывести: класс и число потопленных кораблей.

select
  class
  , SUM(CASE WHEN result='sunk' THEN 1 ELSE 0 END) as sunks
  from (
    select c.class, name from classes c
      left join ships s on c.class=s.class
    union
    select class, ship from classes
      join outcomes on class=ship
  ) as sh
  left join outcomes o on sh.name=o.ship
  group by class

    
57
Для классов, имеющих потери в виде потопленных кораблей и не менее 3 кораблей в базе данных, вывести имя класса и число потопленных кораблей.

select
  class
  , SUM(CASE WHEN result='sunk' THEN 1 ELSE 0 END) as sunks
  from (
    select c.class, name from classes c
      join ships s on c.class=s.class
    union
    select class, ship from classes 
      join outcomes on class=ship
  ) as sh
  left join outcomes o on sh.name=o.ship
  group by class
  having
    SUM(CASE WHEN result='sunk' THEN 1 ELSE 0 END) > 0
    and (select count(si.name) from (
            select s.name, s.class from ships s
            union
            select o.ship, o.ship from outcomes o
          ) as si
        where si.class = sh.class
        group by si.class
        )>=3

    
58
Для каждого типа продукции и каждого производителя из таблицы Product c точностью до двух десятичных знаков найти процентное отношение числа моделей данного типа данного производителя к общему числу моделей этого производителя. Вывод: maker, type, процентное отношение числа моделей данного типа к общему числу моделей производителя

select distinct
  maker, type
  , cast(ROUND((
      count(model) over(partition by maker, type))*100.0/
      count(model) over(partition by maker)
    ,2) as NUMERIC(5,2))
      as 'mods of type / mods total, %'
  from (
    select
      pt.maker, pt.type, p.model
      from (
        select distinct a.maker, b.type
        from product a, product b
      ) pt
      left join product p on pt.maker=p.maker and pt.type=p.type
  ) as p
order by maker, type

    
60
Посчитать остаток денежных средств на начало дня 15/04/01 на каждом пункте приема для базы данных с отчетностью не чаще одного раза в день. Вывод: пункт, остаток. Замечание. Не учитывать пункты, информации о которых нет до указанной даты.

select
  coalesce(i.point,o.point) as point
  ,sum(coalesce(inc,0))-sum(coalesce(out,0)) as remain
  from income_o i
  full join outcome_o o on i.date=o.date and i.point=o.point
  where coalesce(i.date,o.date) < '2001-04-15'
group by coalesce(i.point,o.point)
order by 1,2

    
61
Посчитать остаток денежных средств на всех пунктах приема для базы данных с отчетностью не чаще одного раза в день.

select
  sum(coalesce(inc,0))-sum(coalesce(out,0)) as remain
  from income_o i
  full join outcome_o o on i.date=o.date and i.point=o.point

    
    
62
Посчитать остаток денежных средств на всех пунктах приема на начало дня 15/04/01 для базы данных с отчетностью не чаще одного раза в день.

select
  sum(coalesce(inc,0))-sum(coalesce(out,0)) as remain
  from income_o i
  full join outcome_o o on i.date=o.date and i.point=o.point
  where coalesce(i.date,o.date) < '2001-04-15'

    
63
Определить имена разных пассажиров, когда-либо летевших на одном и том же месте более одного раза.

select name from passenger
where id_psg in (
  select
    p.id_psg
    from pass_in_trip p
    group by p.id_psg, p.place
    having count(*) > 1
)

    
64
Используя таблицы Income и Outcome, для каждого пункта приема определить дни, когда был приход, но не было расхода и наоборот. Вывод: пункт, дата, тип операции (inc/out), денежная сумма за день.

select
  coalesce(i.point,o.point) as point
  ,coalesce(i.date,o.date) as date
  ,CASE WHEN sum(inc) is not null
        THEN 'inc' ELSE 'out'
   END as operation
  ,CASE WHEN sum(inc) is not null
        THEN sum(inc)
        ELSE sum(out)
    END as money
  from income i
  full join outcome o on i.date=o.date and i.point=o.point
  group by coalesce(i.point,o.point), coalesce(i.date,o.date)
  having sum(inc) is null OR sum(out) is null
order by 1,2

    

67
Найти количество маршрутов, которые обслуживаются наибольшим числом рейсов. Замечания.

A - B и B - A считать РАЗНЫМИ маршрутами.
Использовать только таблицу Trip
with q as (
  -- подзапрос считает кол-во рейсов
  -- для каждого направления {town_from, town_to}
  select
    count(*) as c
    from trip
  group by town_from, town_to
)

select count(*) as route_count from q
  where c=(select max(c) from q)

    
68
Найти количество маршрутов, которые обслуживаются наибольшим числом рейсов. Замечания:

A - B и B - A считать ОДНИМ И ТЕМ ЖЕ маршрутом.
Использовать только таблицу Trip
with rc as (
  select
    count(*) as route_trips
    from trip
  group by
    case when town_from > town_to
          then town_from else town_to
    end
    ,case when town_from < town_to
          then town_from else town_to
    end
)
select count(*) as route_count from rc
where route_trips=(select max(route_trips) from rc);


69
По таблицам Income и Outcome для каждого пункта приема найти остатки денежных средств на конец каждого дня, в который выполнялись операции по приходу и/или расходу на данном пункте. Учесть при этом, что деньги не изымаются, а остатки/задолженность переходят на следующий день. Вывод: пункт приема, день в формате "dd/mm/yyyy", остатки/задолженность на конец этого дня.

with q as (
  select
    isnull(i.point, o.point) point
    , isnull(i.date, o.date) date
    , coalesce(sum(i.inc), 0) - coalesce(sum(o.out), 0) balance
    from income i
    full join outcome o
      on i.point=o.point and i.date=o.date and i.code=o.code
    group by isnull(i.point, o.point), isnull(i.date, o.date)
)
select
  point
  , convert(varchar, date, 103) day
  , sum(balance) over(partition by point order by date RANGE UNBOUNDED PRECEDING) as rem
  from q
order by point,date
    

    
70
Укажите сражения, в которых участвовало по меньшей мере три корабля одной и той же страны.

SELECT
  DISTINCT battle
  FROM (
    SELECT
      battle, country
      FROM (
        SELECT battle, country
          FROM Outcomes INNER JOIN Classes ON ship = class
          where ship not in (select name from ships)
        UNION ALL
        SELECT battle, country
          FROM Outcomes o
            INNER JOIN Ships s ON o.ship = s.name
            INNER JOIN Classes c ON s.class = c.class
    ) x
    GROUP BY battle, country
    HAVING COUNT(*) >= 3
  ) y

    
71
Найти тех производителей ПК, все модели ПК которых имеются в таблице PC.

select
  p.maker
  from Product p left join PC on p.model=pc.model
  where p.type='pc'
group by maker
having count(p.model)=count(pc.model);

    
73
Для каждой страны определить сражения, в которых не участвовали корабли данной страны. Вывод: страна, сражение

select country, name as battle from classes, battles
except
select country, battle
from (
  select class, name as ship_name from ships
  union
  select ship, ship from outcomes
) as sh
join Classes c on sh.class=c.class
join Outcomes o on o.ship=sh.ship_name;

    
74
Вывести классы всех кораблей России (Russia). Если в базе данных нет классов кораблей России, вывести классы для всех имеющихся в БД стран. Вывод: страна, класс

select
  country, class
  from Classes
  -- We can replace A XOR B operation with this
  -- (A and !B) OR (!A and B)
  where (country='russia' and 'russia'=ANY(select country from Classes))
      OR (country!='russia' and NOT ('russia' = ANY(select country from Classes)))


75
Для каждого корабля из таблицы Ships указать название первого по времени сражения из таблицы Battles, в котором корабль мог бы участвовать после спуска на воду.

Если год спуска на воду неизвестен, взять последнее по времени сражение.
Если нет сражения, произошедшего после спуска на воду корабля, вывести NULL вместо названия сражения.
Считать, что корабль может участвовать во всех сражениях, которые произошли в год спуска на воду корабля.

Вывод: имя корабля, год спуска на воду, название сражения

Замечание: считать, что не существует двух битв, произошедших в один и тот же день.

with q as (
  select
    s.name, launched, b.name as b_name
    , rank() over(partition by s.name, launched order by date asc) as num
  from ships s left join battles b on datepart(yyyy,date)>=s.launched
)
select
  name, launched
  ,
    case
      when launched is null
      then (select name from battles where date=(select max(date) from battles))

      when launched is not null and launched>(select datepart(yyyy,max(date)) from battles)
      then NULL

      else b_name
    end
   as battle
from q
where num=1
